In [1]:
import selenium
import pandas as pd
import numpy as np
import time
import requests
import re
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException, ElementClickInterceptedException

import warnings
warnings.filterwarnings('ignore')

In [2]:
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(options=options, executable_path='chromedriver.exe')
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                                                                     'AppleWebKit/537.36 (KHTML, like Gecko) '
                                                                     'Chrome/85.0.4183.102 Safari/537.36'})

{}

In [3]:
url = "https://www.yatra.com/"
driver.get(url)
time.sleep(3)

In [4]:
driver.find_element_by_xpath("/html/body/div[2]/div/section/div[1]/div/div[1]/section/div/div/div/div[1]/div[3]/div[2]/div[1]/input").click()
time.sleep(3)

In [5]:
loc_lst = ['New Delhi', 'Mumbai', 'Bangalore', 'Chennai','Hyderabad ','Goa ','Kolkata ','Jaipur ','Lucknow ']

In [6]:
Airline_name=[]      # The name of the airline.
Departure_time = []  # The time when the journey starts from the source.
Arrival_time = []    # Time of arrival at the destination.
Duration=[]          # Total duration taken by the flight to reach the destination from the source.
Stops = []           # Total stops between the source and destination.
Source = []          # The source from which the service begins.
Destination = []     # The destination where the service ends.
Meal=[]              # Availability of meals in the flight.
Price=[]             # The price of the flight ticket.
Location=[] 

In [7]:
for x in loc_lst:
    for y in loc_lst:
        if x!=y:                 
            Source = driver.find_element_by_xpath("//div[@class='input-holder pb-2 bdr-btm']/input")
            time.sleep(2)
            Source.clear() 
            Source.send_keys(x)
            time.sleep(2)
            
            # Fetching webelement of destination
            Destination = driver.find_element_by_xpath("//div[@class='input-holder  bdr-btm pb-2']/input")
            Destination.click()
            Destination.clear()
            time.sleep(2)
            Destination.send_keys(y)
            time.sleep(2)
            
            try:
                driver.find_element_by_xpath("/html/body/section[2]/section/div[1]/div/div[1]").click()
            except NoSuchElementException:
                pass
            # Searching for flights again and clicking in search again button
            try:
                srch_btn = driver.find_element_by_xpath("//button[@class='fs-14 btn-submit cursor-pointer bold']")
                wait = WebDriverWait(driver, 10)
                wait.until(EC.visibility_of(srch_btn))
                srch_btn.click()
            except NoSuchElementException:
                pass
            time.sleep(2)
            
            # Scrolling the data 5000 times 
            for _ in range(15):
                time.sleep(1)
                driver.execute_script("window.scrollBy(0,5000)")
                time.sleep(5)
            
            # Fetching web element of scroll to top button
            try:
                driver.find_element_by_xpath("//div[@title='scroll to top']").click()
            except NoSuchElementException:
                pass
            
             # Fetching name of Airline
            try:
                for i in driver.find_elements_by_xpath("//div[@class='fs-13 airline-name no-pad col-8']/span"):
                    Airline_name.append(i.text)
            except NoSuchElementException:
                pass
            
            # Fetching departure time of the flights
            try:
                for i in driver.find_elements_by_xpath("//div[@class='i-b pr']"):
                    Departure_time.append(i.text)
            except NoSuchElementException:
                pass
            
            # Fetching arrival time of the flights 
            try:
                for i in driver.find_elements_by_xpath("//div[@class='i-b pdd-0 text-left atime col-5']//p[1]"):
                    Arrival_time.append(i.get_attribute("innerHTML").splitlines()[0][0:5])
            except NoSuchElementException:
                pass
            
            # Fetching Duration of flights journey
            try:
                for i in driver.find_elements_by_xpath("//div[@class='stop-cont pl-13']/p"):
                    Duration.append(i.text)
            except NoSuchElementException:
                pass
            
            # Fetching number of stops the flights have between source and destination
            try:
                for i in driver.find_elements_by_xpath("//div[@class=' font-lightgrey fs-10 tipsy i-b fs-10']/span[1]"):
                    Stops.append(i.text)
            except NoSuchElementException:
                pass
            
            # Fetching availability of meal in flights
            try:
                for i in driver.find_elements_by_xpath("//div[@class='features pull-right fs-12 flex']/div[1]"):
                    Meal.append(i.text)
            except NoSuchElementException:
                pass
            
            # Fetching Locations
            try:
                for i in driver.find_elements_by_xpath("//p[@class='fs-10 font-lightgrey no-wrap city ellipsis']"):
                    Location.append(i.text)
            except NoSuchElementException:
                pass
            
            # Fetching Prices of flight tickets
            try:
                for i in driver.find_elements_by_xpath("//div[@class='i-b tipsy fare-summary-tooltip fs-18']"):
                    Price.append(i.text)
            except NoSuchElementException:
                pass

In [8]:
Source = Location[0:len(Location):2]
Destination = Location[1:len(Location):2]

In [9]:
len(Source), len(Destination)

(4004, 4004)

In [10]:
print(len(Price),len(Airline_name),len(Departure_time),len(Arrival_time),len(Duration),len(Stops),len(Meal))

4004 3664 4004 4004 4004 4004 4004


In [11]:
data = list(zip(Airline_name,Departure_time,Arrival_time,Duration,Source,Destination,Meal,Stops,Price))
df = pd.DataFrame(data, columns = ["Airline",
                                   "Departure_time",
                                   "Time_of_arrival",
                                   "Duration", 
                                   "Source",
                                   "Destination",
                                   "Meal_availability",
                                   "Number_of_stops",
                                   "Price"])
df

,Airline,Departure_time,Time_of_arrival,Duration,Source,Destination,Meal_availability,Number_of_stops,Price
0,IndiGo,11:55,13:55,2h 00m,New Delhi,Mumbai,Emissions: 142 Kg CO2,Non Stop,"7,319"
1,IndiGo,10:45,12:50,2h 05m,New Delhi,Mumbai,Emissions: 142 Kg CO2,Non Stop,"7,319"
2,IndiGo,17:00,19:05,2h 05m,New Delhi,Mumbai,eCash 250,Non Stop,"7,319"
3,IndiGo,23:35,01:40,2h 05m,New Delhi,Mumbai,eCash 250,Non Stop,"7,319"
4,IndiGo,02:00,04:10,2h 10m,New Delhi,Mumbai,Emissions: 142 Kg CO2,Non Stop,"7,319"
...,...,...,...,...,...,...,...,...,...
3659,Air India,06:10,21:55,15h 45m,Jaipur,Kolkata,eCash 250,1 Stop,"10,614"
3660,Air India,20:50,01:35,4h 45m,Jaipur,Kolkata,eCash 250,1 Stop,"11,726"
3661,Air India,14:20,20:45,6h 25m,Jaipur,Kolkata,eCash 250,1 Stop,"11,726"
3662,Air India,22:25,05:05,6h 40m,Jaipur,Kolkata,eCash 250,1 Stop,"11,726"


In [12]:
# Saving the data into excel file
df.to_excel("Flight_Prices.xlsx")

I have successfully scraped the required data from the website yatra.com. The dataframe consists of 5303 rows and 9 columns. And I have saved this dataframe in excel format as "Flight_Prices".

## Done